# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,papatowai,-46.5619,169.4708,284.59,79,28,2.58,NZ,1686620685
1,1,nchelenge,-9.3451,28.7340,291.92,60,28,2.38,ZM,1686620685
2,2,tazovsky,67.4667,78.7000,273.96,93,100,8.12,RU,1686620685
3,3,mount gambier,-37.8333,140.7667,285.07,82,100,4.88,AU,1686620685
4,4,olonkinbyen,70.9221,-8.7187,275.20,87,100,8.86,SJ,1686620686


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng","Lat", 
                                      geo = True, color = "City",
                                      alpha = 0.6, size = "Humidity",
                                      tiles = "OSM"
                                    )
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_df = city_data_df.loc[(city_data_df["Humidity"] < 20)
                                      ]

# Drop any rows with null values
filtered_df.dropna()

# Display sample data
filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
90,90,turpan,42.9333,89.1667,308.24,16,100,3.17,CN,1686620709
135,135,novyy turtkul',41.5500,61.0167,293.10,17,3,5.11,UZ,1686620721
190,190,mount isa,-20.7333,139.5000,301.02,18,7,4.63,AU,1686620738
199,199,as sulayyil,20.4607,45.5779,305.29,18,100,6.01,SA,1686620740
220,220,illizi,26.4833,8.4667,304.04,17,0,4.88,DZ,1686620745
240,240,korla,41.7597,86.1469,305.18,19,97,5.32,CN,1686620751
256,256,saynshand,44.8824,110.1163,299.94,18,8,3.05,MN,1686620756
319,319,qaisumah,28.3112,46.1273,304.13,18,0,2.06,SA,1686620773
344,344,crane,31.3974,-102.3501,306.74,12,1,4.48,US,1686620781
369,369,anar darah,32.7587,61.6540,300.01,15,0,4.31,AF,1686620788


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
90,turpan,CN,42.9333,89.1667,16,
135,novyy turtkul',UZ,41.5500,61.0167,17,
190,mount isa,AU,-20.7333,139.5000,18,
199,as sulayyil,SA,20.4607,45.5779,18,
220,illizi,DZ,26.4833,8.4667,17,
240,korla,CN,41.7597,86.1469,19,
256,saynshand,MN,44.8824,110.1163,18,
319,qaisumah,SA,28.3112,46.1273,18,
344,crane,US,31.3974,-102.3501,12,
369,anar darah,AF,32.7587,61.6540,15,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel","apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    hot_lon = row["Lng"]
    hot_lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{hot_lon},{hot_lat},{radius}"
    params["bias"] = f"proximity:{hot_lon},{hot_lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
turpan - nearest hotel: Oriental
novyy turtkul' - nearest hotel: To'rtku'l miymanxanasi
mount isa - nearest hotel: Ibis Styles
as sulayyil - nearest hotel: No hotel found
illizi - nearest hotel: بوناقة
korla - nearest hotel: Silver Star Hotel
saynshand - nearest hotel: No hotel found
qaisumah - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
anar darah - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
ruidoso - nearest hotel: Hotel Ruidoso
alamogordo - nearest hotel: Days Inn Alamogordo
gowurdak - nearest hotel: No hotel found
kandahar - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
90,turpan,CN,42.9333,89.1667,16,Oriental
135,novyy turtkul',UZ,41.5500,61.0167,17,To'rtku'l miymanxanasi
190,mount isa,AU,-20.7333,139.5000,18,Ibis Styles
199,as sulayyil,SA,20.4607,45.5779,18,No hotel found
220,illizi,DZ,26.4833,8.4667,17,بوناقة
240,korla,CN,41.7597,86.1469,19,Silver Star Hotel
256,saynshand,MN,44.8824,110.1163,18,No hotel found
319,qaisumah,SA,28.3112,46.1273,18,No hotel found
344,crane,US,31.3974,-102.3501,12,No hotel found
369,anar darah,AF,32.7587,61.6540,15,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.75,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)